In [1]:
import sys
sys.path.append('/media/arclabdl1/HD1/Linjun/mpc-mpnet-py/deps/sparse_rrt-1')
from sparse_rrt import _sst_module
from sparse_rrt.systems import standard_cpp_systems

import numpy as np
import time

In [2]:
# from sparse_rrt.experiments.experiment_utils import run_config
# point_config = dict(
#     system='point',
#     planner='sst_backend',
#     start_state=[9., 9.],
#     goal_state=[-9., 9.],
#     goal_radius=0.5,
#     random_seed=100,
#     integration_step=0.02,
#     min_time_steps=2,
#     max_time_steps=20,
#     number_of_iterations=300000,
#     display_type=None,
#     sst_delta_near=0.2,
#     sst_delta_drain=0.2
# )
# run_config(point_config)


def test_sst_acrobot(min_time_steps, max_time_steps, integration_step, number_of_iterations):
    '''
    Test that runs are deterministic
    '''
    system = standard_cpp_systems.TwoLinkAcrobot()

    def _create_planner(system):
        return _sst_module.SSTBackendWrapper(
            state_bounds=system.get_state_bounds(),
            control_bounds=system.get_control_bounds(),
            distance=system.distance_computer(),
            start_state=np.array([0., 0., 0, 0,]),
            goal_state=np.array([3, 0, 0, 0]),
            goal_radius=10,
            random_seed=0,
            sst_delta_near=0.2,
            sst_delta_drain=0.2
        )
    planner = _create_planner(system)
    start_time = time.time()

    for iteration in range(number_of_iterations):
        planner.step(system, min_time_steps, max_time_steps, integration_step)
        solution = planner.get_solution()
        if iteration % 1000 == 0:
            if solution is None:
                solution_cost = None
            else:
                solution_cost = np.sum(solution[2])

            print("Time: %.2fs, Iterations: %d, Nodes: %d, Solution Quality: %s" %
                  (time.time() - start_time, iteration, planner.get_number_of_nodes(), solution_cost))
    return planner

p = test_sst_acrobot(1, 100, 2e-2, 2000)

Time: 0.00s, Iterations: 0, Nodes: 2, Solution Quality: None
Time: 0.37s, Iterations: 1000, Nodes: 917, Solution Quality: 14.700000000000003


In [3]:
p.nearest_vertex(np.array([0, 0, 0, 0]))

array([0., 0., 0., 0.])

In [4]:
p.add_to_tree(np.array([3, 0, 0, 0]), 6.2)

In [6]:
p.get_solution()

(array([[0., 0., 0., 0.],
        [3., 0., 0., 0.]]),
 array([[4.65016409e-310]]),
 array([6.2]))